# Project 4 - Part 2: Cleaning and feature extraction

# 2.1 Importing and loading data

# 2.1.1 Import packages

In [174]:
#Importing necssary packages
import pandas as pd
import numpy as np
import re
import string

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 2.1.2 Loading and inspecting data

In [175]:
#Read the job data csv created in the webscraping notebook
file = 'jobs.csv'
jobs = pd.read_csv(file)
jobs.head()

,company,location,salary,summary,title
0,Allianz,Melbourne VIC,"Full-time, Temporary",Reporting directly to the Manager Business Ana...,DATA SCIENTIST
1,Teradata,Melbourne VIC,No info,The Data Scientist produces intelligence from ...,Data Scientist
2,carsales.com.au,Melbourne VIC,"Full-time, Permanent",We are currently on the hunt for a Data Scient...,Data Scientist - 12 month contract
3,EY,Melbourne VIC,No info,About us\r\nEY DnA is the data and advanced an...,Data Scientist - Melbourne
4,Culture Amp,Melbourne VIC,No info,Culture Amp is the world's most powerfully sim...,Data Scientist


In [176]:
#Inspect data types
jobs.dtypes

company     object
location    object
salary      object
summary     object
title       object
dtype: object

# 2.2 Extracting and cleaning data

# 2.2.1 Target variable: salary

In [177]:
#Inspecting for nulls
jobs.isnull().sum()

company     0
location    0
salary      0
summary     0
title       0
dtype: int64

In [178]:
#Change those 'No info' values in salary to NaN
jobs['salary'] = jobs['salary'].apply(lambda x: np.nan if x == 'No info' else x)

In [179]:
#What columns can we pull out from the salary column?
jobs['salary'].unique()[:20]

array(['Full-time,\xa0Temporary', nan, 'Full-time,\xa0Permanent',
       '\r\n                $110,000 - $150,000 a year',
       '\r\n                $100,000 - $150,000 a year',
       '\r\n                $95,000 - $130,000 a year - ',
       '\r\n                $93,819 a year',
       '\r\n                $110,000 - $130,000 a year - ',
       '\r\n                $71,418 - $75,731 a year',
       '\r\n                $160,000 a year - ',
       '\r\n                $97,000 - $105,000 a year',
       '\r\n                $100,000 - $150,000 a year - ',
       '\r\n                $76,468 - $104,260 a year', 'Contract',
       'Part-time,\xa0Temporary,\xa0Internship,\xa0Volunteer',
       'Temporary,\xa0Contract', '\r\n                $1,000 a day - ',
       '\r\n                $111,512 a year',
       '\r\n                $65,000 - $75,000 a year',
       '\r\n                $135,000 - $220,000 a year - '], dtype=object)

In [180]:
#Create regular expressions to extract the low and high numbers from jobs that list an annual salary range, return same for both
#columns if only one value is listed
low_re = '\$\d\S+\s\-'
high_re = '\-\s\$\d\S+\s'
other_re = '[^-]\s\$\d\S+\s(?!-)'

jobs['salary_low'] = [re.findall(low_re, str(i)) if len(re.findall(low_re, str(i))) > 0 else re.findall(other_re, str(i)) for i in jobs['salary']]
jobs['salary_high'] = [re.findall(high_re, str(i)) if len(re.findall(high_re, str(i))) > 0 else re.findall(other_re, str(i)) for i in jobs['salary']]

In [181]:
#Now get the values out of lists, remove unwanted characters and convert columns to floats
unwanted = '$,- '
removechars = str.maketrans('','',unwanted)
for i in range(len(jobs['salary_low'])):
    try:
        jobs['salary_low'][i] = jobs['salary_low'][i][0].translate(removechars)
    except:
        jobs['salary_low'][i] = np.nan
for i in range(len(jobs['salary_high'])):
    try:
        jobs['salary_high'][i] = jobs['salary_high'][i][0].translate(removechars)
    except:
        jobs['salary_high'][i] = np.nan
        
jobs['salary_low'] = jobs['salary_low'].astype('float')
jobs['salary_high'] = jobs['salary_high'].astype('float')

In [182]:
#There are quite a few duplicate rows, possibly ones that came up in both searches I conducted in the scraping. 
#I'll drop duplicate rows
jobs = jobs.drop_duplicates()
jobs = jobs.reset_index()
jobs.shape

(478, 8)

In [183]:
jobs.describe()

,index,salary_low,salary_high
count,478.000000,98.000000,98.000000
mean,389.163180,84498.561224,100455.948980
std,232.441987,41863.902360,48546.461248
min,0.000000,39.000000,40.000000
25%,195.250000,70000.000000,81060.250000
50%,366.500000,85000.000000,98748.500000
75%,589.750000,100000.000000,130000.000000
max,796.000000,190000.000000,220000.000000


In [184]:
#Some of the salary information is listed by month
jobs[jobs['salary'].str.contains('month')==True]

,index,company,location,salary,summary,title,salary_low,salary_high


In [128]:
#We'll convert these to annual rates by multiplying by 12
mask = jobs[jobs['salary'].str.contains('month')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * 12)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * 12)

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [129]:
#Some are weekly
jobs[jobs['salary'].str.contains('week')==True]

,index,company,location,salary,summary,title,salary_low,salary_high
130,209,CSIRO,Australia,\r\n $590 a week,Improve your professional employment prospects...,Indigenous Cadetships - Data61 Computer Science,590.0,590.0


In [130]:
#We'll just convert these to something annual by multiplying by 52
mask = jobs[jobs['salary'].str.contains('week')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * 52)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * 52)

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [131]:
#Some by day
jobs[jobs['salary'].str.contains('day')==True]

,index,company,location,salary,summary,title,salary_low,salary_high
55,89,Total Resource Solutions,Melbourne VIC,"\r\n $1,000 a day -",Our client runs one of the largest datasets in...,Data Engineer - AWS,1000.0,1000.0
114,184,Hudson,Sydney NSW,\r\n $600 - $850 a day -,Data Scientist - 6 month Contract\r\n$600-$850...,Data Scientist - 6 month Contract,600.0,850.0
193,298,AUREC,Sydney NSW,\r\n $600 a day -,Where you’ll be working:\r\n\r\nYou will be wo...,Data Engineer: (GCP/Big Data/ML) Amazing Oppor...,600.0,600.0
196,304,Hudson,Brisbane QLD,\r\n $800 - $850 a day -,Agile Technical Business Analyst required | Pr...,Product Analyst,800.0,850.0
345,572,Charterhouse,New South Wales,\r\n $450 a day -,Job details\r\nLocation: NSW\r\n\r\nSalary: Up...,Data Analyst| North Sydney| Temporary 3 months,450.0,450.0
350,580,Cigniti Technologies,Darwin NT,\r\n $500 - $550 a day -,Job SummaryGreetings from Cigniti Technologies...,Business Analyst-Healthcare,500.0,550.0
418,697,Talenza,Sydney NSW,\r\n $700 - $800 a day -,Data Analyst\r\n\r\nLeading FS FinTech Applica...,Data Analyst,700.0,800.0


In [132]:
#We'll convert these to an annual rate by multiplying by the number of working days in a year. I'll assume that this is
#effectively 5 * 52.
workingdays = 5 * 52
mask = jobs[jobs['salary'].str.contains('day')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * workingdays)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * workingdays)

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [133]:
#Some per hour
jobs[jobs['salary'].str.contains('hour')==True]

,index,company,location,salary,summary,title,salary_low,salary_high
186,289,Chandler Macleod,Sydney NSW,\r\n $39 - $40 an hour -,Location: Sydney CBD\r\nContract: 6 months\r\n...,Ecohydrologist,39.0,40.0
275,440,Project Professional Services,Melbourne VIC,\r\n $60 an hour -,About the roleThe role involes obtaining requi...,Reporting Analyst,60.0,60.0
476,795,HorizonOne Recruitment,Canberra ACT,\r\n $50 - $60 an hour -,The Organisation:\r\nOur client is a central G...,APS6 Data Modeller/Analyst,50.0,60.0
477,796,Profusion Group,Sydney NSW,\r\n $50 - $60 an hour,Company Overview\r\n\r\nJoin an iconic financi...,Data Analyst (GST),50.0,60.0


In [134]:
#These ones we'll convert to an annual rate by assuming 8 hours a day, for the same number of annual working days as above
mask = jobs[jobs['salary'].str.contains('hour')==True].index
jobs['salary_low'][mask] = jobs['salary_low'][mask].apply(lambda x: x * workingdays * 8)
jobs['salary_high'][mask] = jobs['salary_high'][mask].apply(lambda x: x * workingdays * 8)

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [135]:
#Since we have a range of salary values for many of the jobs, I'll summarize these two range values in an extra column
#The mean salary
jobs['salary_mean'] = (jobs['salary_low'] + jobs['salary_high']) / 2

In [136]:
#Now we can look at the stats for the salary again
jobs.describe()
#Looks better! Now I will start working on pulling out some feature columns.

,index,salary_low,salary_high,salary_mean
count,478.000000,98.000000,98.000000,98.000000
mean,389.163180,101316.530612,118908.704082,110112.617347
std,232.441987,36850.461484,39782.587953,37320.319603
min,0.000000,30680.000000,30680.000000,30680.000000
25%,195.250000,76471.500000,90000.000000,83495.875000
50%,366.500000,94300.500000,116138.000000,105000.000000
75%,589.750000,119250.000000,142250.000000,128750.000000
max,796.000000,260000.000000,260000.000000,260000.000000


# 2.2.2 Job title keywords

In [137]:
#Let's start by creating some categories from the job title column
#Our bag-of-words will be the 'title' column of our jobs dataframe
bow = jobs['title']

In [138]:
#Fit a Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tvec = TfidfVectorizer(stop_words='english')
tvec.fit(bow)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [139]:
#Generate columns from the count vectorizer
tvecdf  = pd.DataFrame(tvec.transform(bow).todense(),
             columns=tvec.get_feature_names())

tvecdf.transpose().sort_values(0, ascending=False).transpose().head()

,scientist,data,portfolio,plethora,platforms,platform,planning,planners,php,phenotyping,...,experimental,executive,excel,equity,equities,environmental,enterprise,english,engineers,year
0,0.799049,0.601265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.799049,0.601265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.268734,0.202216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429897,0.323487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.799049,0.601265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
#We probably don't want to include *all* the words as features, so I will reorder the columns by how many times each word
#appears in the corpus
tvecdf = tvecdf.reindex_axis(tvecdf.sum().sort_values(ascending=False).index, axis=1)
tvecdf.head().iloc[:,:30]

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,data,analyst,scientist,senior,engineer,business,research,analytics,insights,learning,...,graduate,support,sydney,manager,test,melbourne,specialist,contract,systems,intelligence
0,0.601265,0.0,0.799049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,0.601265,0.0,0.799049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.202216,0.0,0.268734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.505675,0.0,0.0
3,0.323487,0.0,0.429897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.842938,0.0,0.000000,0.0,0.0
4,0.601265,0.0,0.799049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [141]:
tvecdf.shape

(478, 475)

In [142]:
#Let's rename the columns so we know where they came from later
tvecdf.rename(columns = lambda x: 'title_'+str(x), inplace=True)

In [143]:
#Let's add the 20 most common terms to the jobs dataframe
top30 = tvecdf.iloc[:,:30]
jobs = jobs.join(top30)

In [144]:
jobs.head(2)

,index,company,location,salary,summary,title,salary_low,salary_high,salary_mean,title_data,...,title_graduate,title_support,title_sydney,title_manager,title_test,title_melbourne,title_specialist,title_contract,title_systems,title_intelligence
0,0,Allianz,Melbourne VIC,"Full-time, Temporary",Reporting directly to the Manager Business Ana...,DATA SCIENTIST,NaN,NaN,NaN,0.601265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Teradata,Melbourne VIC,NaN,The Data Scientist produces intelligence from ...,Data Scientist,NaN,NaN,NaN,0.601265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2.2.3 Location

In [145]:
#Now to make some location-based columns
#First, note that we have a lot of variation in how particular locations are listed in the data. Some places are listed as 
#a city, such as Sydney. Others show the suburb. Almost all however, show a state. I will create state-level location
#columns from this column
jobs['location'].value_counts().head(20)

Sydney NSW                              151
Melbourne VIC                           124
Australia                                18
Brisbane QLD                             18
New South Wales                          14
Melbourne City Centre VIC                13
Perth WA                                 12
Sydney Central Business District NSW     12
Canberra ACT                             11
Adelaide SA                              10
Queensland                                6
Victoria                                  5
Pyrmont NSW                               4
Darlinghurst NSW                          4
Newcastle NSW                             3
Macquarie Park NSW                        3
South Melbourne VIC                       2
Western Australia                         2
Docklands VIC                             2
Parkville VIC                             2
Name: location, dtype: int64

In [146]:
#Use a regex to pull out any strings of 2 or 3 capital letters (like in 'NSW' or 'SA'), or return the whole string if no such 
#string is present
state_re = '\s[A-Z]{2,3}'
jobs['state'] = [re.findall(state_re, str(i)) if len(re.findall(state_re, str(i))) > 0 else i for i in jobs['location']]

In [147]:
#Convert any lists to strings
for i in jobs.index:
    if len(jobs['state'][i]) < 4:
        jobs['state'][i] = jobs['state'][i][0]
    else:
        jobs['state'][i] = jobs['state'][i]

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [148]:
#Remove spaces
jobs['state'] = jobs['state'].apply(lambda x: x.replace(' ',''))

In [149]:
#Check unique values
jobs['state'].unique()

array(['VIC', 'NSW', 'SA', 'QLD', 'Australia', 'WA', 'ACT',
       'NewSouthWales', 'Tasmania', 'Queensland', 'Victoria',
       'WesternAustralia', 'SouthAustralia', 'NT', 'TAS'], dtype=object)

In [150]:
#Convert full length names to abbreviations
jobs['state'] = jobs['state'].map({'Queensland':'QLD','NewSouthWales':'NSW','Australia':'AUS','Victoria':'VIC',
                                  'WesternAustralia':'WA','SouthAustralia':'SA','Tasmania':'TAS',
                                  'NSW':'NSW','QLD':'QLD','WA':'WA','VIC':'VIC','ACT':'ACT','SA':'SA', 'NT':'NT'})

In [151]:
#Check unique values again
jobs['state'].unique()

array(['VIC', 'NSW', 'SA', 'QLD', 'AUS', 'WA', 'ACT', 'TAS', 'NT', nan],
      dtype=object)

In [152]:
#Make dummy columns for states
state_dummies = pd.get_dummies(jobs['state'])
state_dummies.rename(columns = lambda x: 'state_'+str(x), inplace=True)
state_dummies.sample(5)

,state_ACT,state_AUS,state_NSW,state_NT,state_QLD,state_SA,state_TAS,state_VIC,state_WA
259,0,0,0,0,1,0,0,0,0
296,0,0,1,0,0,0,0,0,0
127,0,0,1,0,0,0,0,0,0
306,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,1,0


In [153]:
#Add them to the dataframe
jobs = jobs.join(state_dummies)

# 2.2.4 Summary keywords

In [154]:
#First clean up unwanted characters such as newline and - characters
unwanted = '\r\n-'
removechars = str.maketrans(' ',' ',unwanted)
jobs['summary'] = jobs['summary'].apply(lambda x: x.translate(removechars))

In [155]:
#I will look for important summary keywords using a Tf-idf vectorizer
from sklearn.feature_extraction import text
from nltk.stem import PorterStemmer, SnowballStemmer

I've elected to do stemming before extracting words from the summary since it is more likely that different variants of the same 
word will appear in the summary than the job title.

In [156]:
#First I'll do some stemming to take care of redundant word endings (e.g. 'experience' vs 'experienced')
stemmer = SnowballStemmer('english')
jobs['stemmed_summary'] = [[stemmer.stem(word) for word in sentence.split(" ")] for sentence in jobs['summary']]

In [157]:
for i in jobs['stemmed_summary'].index:
    jobs['stemmed_summary'][i] = str(jobs['stemmed_summary'][i])

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [158]:
stop_words = text.ENGLISH_STOP_WORDS.union(['work','busi','role','look','build','strong','understand','grow','ensur',
                                           'join','key','requir','year','help','www','weeks','deal','mid','exist','datayou'])
tvec = TfidfVectorizer(stop_words=stop_words)
tvec.fit(jobs['stemmed_summary'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=frozenset({'due', 'afterwards', 'thereupon', 'whence', 'about', 'rather', 'see', 'hence', 'hers', 'latter', 'next', 'something', 'bottom', 'as', 'where', 'why', 'found', 'your', 'those', 'further', 'ensur', 'she', 'himself', 'anyone', 'down', 'ltd', 'wherever', 'herself', 'mostly', 'build...e', 'is', 'latterly', 'much', 'strong', 'hasnt', 'behind', 'thereafter', 'eg', 'around', 'against'}),
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [159]:
tvecdf  = pd.DataFrame(tvec.transform(jobs['stemmed_summary']).todense(),
                   columns=tvec.get_feature_names())

tvecdf.transpose().sort_values(0, ascending=False).head(10).transpose()
#Order columns based on the total sum of tf-idf scores
tvecdf = tvecdf.reindex_axis(tvecdf.mean().sort_values(ascending=False).index, axis=1)

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  


In [160]:
tvecdf.shape

(478, 14715)

In [161]:
#Some of these seem like sensible words to look at (like, business, analytics, research, customer etc.). While others
#are likely to be uninformative (e.g work, role, ability) I will look through the top 30 and manually drop columns that seem
#uninformative.
tvecdf.head().iloc[:,:20]

,data,experi,team,analyt,develop,manag,support,custom,product,client,research,provid,project,skill,market,report,applic,includ,learn,opportun
0,0.105371,0.028535,0.058292,0.119263,0.000000,0.036787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108110,0.000000,0.090630,0.000000,0.000000,0.097039,0.000000
1,0.271705,0.010900,0.044536,0.091119,0.012317,0.000000,0.014585,0.034530,0.051795,0.072702,0.021902,0.028498,0.104140,0.027533,0.000000,0.000000,0.031502,0.028899,0.000000,0.000000
2,0.175608,0.038044,0.038859,0.185509,0.021494,0.000000,0.050905,0.150642,0.060257,0.000000,0.000000,0.024865,0.030288,0.048046,0.067751,0.060416,0.000000,0.050430,0.064689,0.073232
3,0.157049,0.034023,0.069504,0.094802,0.019222,0.021932,0.068287,0.000000,0.000000,0.028365,0.000000,0.044475,0.027087,0.021484,0.000000,0.000000,0.024582,0.045100,0.028926,0.021831
4,0.200935,0.090689,0.055579,0.025269,0.020495,0.000000,0.000000,0.028728,0.114912,0.000000,0.072887,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030841,0.046552


In [162]:
top1000 = tvecdf.iloc[:,:5000]
top1000.columns

Index(['data', 'experi', 'team', 'analyt', 'develop', 'manag', 'support',
       'custom', 'product', 'client',
       ...
       'biologist', 'employeeowned', 'incitec', 'difficulti', 'agribusi',
       'bfm', 'cor', 'square', 'tankers', 'farm'],
      dtype='object', length=5000)

In [163]:
#Make a full table of keywords
summ_keywords = top1000#.join(extra)
#Ad the prefix kw_ to the column name to indicate that these are summary keywords
summ_keywords.rename(columns = lambda x: 'kw_'+str(x), inplace=True)
summ_keywords.head()

C:\Users\sreej\Anaconda3\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,kw_data,kw_experi,kw_team,kw_analyt,kw_develop,kw_manag,kw_support,kw_custom,kw_product,kw_client,...,kw_biologist,kw_employeeowned,kw_incitec,kw_difficulti,kw_agribusi,kw_bfm,kw_cor,kw_square,kw_tankers,kw_farm
0,0.105371,0.028535,0.058292,0.119263,0.000000,0.036787,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.271705,0.010900,0.044536,0.091119,0.012317,0.000000,0.014585,0.034530,0.051795,0.072702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.175608,0.038044,0.038859,0.185509,0.021494,0.000000,0.050905,0.150642,0.060257,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.157049,0.034023,0.069504,0.094802,0.019222,0.021932,0.068287,0.000000,0.000000,0.028365,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.200935,0.090689,0.055579,0.025269,0.020495,0.000000,0.000000,0.028728,0.114912,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
#Finally, add these keyword columns to the main dataframe
jobs = jobs.join(summ_keywords)

In [165]:
jobs.shape

(478, 5050)

In [166]:
jobs.head()

,index,company,location,salary,summary,title,salary_low,salary_high,salary_mean,title_data,...,kw_biologist,kw_employeeowned,kw_incitec,kw_difficulti,kw_agribusi,kw_bfm,kw_cor,kw_square,kw_tankers,kw_farm
0,0,Allianz,Melbourne VIC,"Full-time, Temporary",Reporting directly to the Manager Business Ana...,DATA SCIENTIST,NaN,NaN,NaN,0.601265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Teradata,Melbourne VIC,NaN,The Data Scientist produces intelligence from ...,Data Scientist,NaN,NaN,NaN,0.601265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,carsales.com.au,Melbourne VIC,"Full-time, Permanent",We are currently on the hunt for a Data Scient...,Data Scientist - 12 month contract,NaN,NaN,NaN,0.202216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,EY,Melbourne VIC,NaN,About usEY DnA is the data and advanced analyt...,Data Scientist - Melbourne,NaN,NaN,NaN,0.323487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Culture Amp,Melbourne VIC,NaN,Culture Amp is the world's most powerfully sim...,Data Scientist,NaN,NaN,NaN,0.601265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2.2.5 Manually creating skill variables

In [167]:
#Manually creating 'skill' columns

In [168]:
jobs['skill_sql'] = [1 if 'sql' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_python'] = [1 if 'python' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_spark'] = [1 if 'spark' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_hadoop'] = [1 if 'hadoop' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_tableau'] = [1 if 'tableau' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_r'] = [1 if 'r' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_excel'] = [1 if 'excel' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_sas'] = [1 if 'sas' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_spss'] = [1 if 'spss' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_aws'] = [1 if 'aws' in x else 0 for x in jobs['stemmed_summary']]
jobs['skill_kafka'] = [1 if 'kafka' in x else 0 for x in jobs['stemmed_summary']]

# 2.3 Final cleaning and exporting

In [169]:
#Now that we've made feature columns, we have some remaining columns that won't be useful for our modelling.
#These includes the remaining salary info, the full summary, the stemmed summary and the state and original location column.
#I will keep job title and company for indexing purposes.
dropcols = ['location','salary','summary','state','stemmed_summary']

In [170]:
jobs = jobs.drop(dropcols, axis=1)

In [171]:
jobs.head()

,index,company,title,salary_low,salary_high,salary_mean,title_data,title_analyst,title_scientist,title_senior,...,skill_python,skill_spark,skill_hadoop,skill_tableau,skill_r,skill_excel,skill_sas,skill_spss,skill_aws,skill_kafka
0,0,Allianz,DATA SCIENTIST,NaN,NaN,NaN,0.601265,0.0,0.799049,0.0,...,1,0,0,0,1,0,1,0,0,0
1,1,Teradata,Data Scientist,NaN,NaN,NaN,0.601265,0.0,0.799049,0.0,...,1,1,1,0,1,1,0,0,0,0
2,2,carsales.com.au,Data Scientist - 12 month contract,NaN,NaN,NaN,0.202216,0.0,0.268734,0.0,...,1,0,0,0,1,1,1,1,0,0
3,3,EY,Data Scientist - Melbourne,NaN,NaN,NaN,0.323487,0.0,0.429897,0.0,...,1,0,0,1,1,0,1,1,0,0
4,4,Culture Amp,Data Scientist,NaN,NaN,NaN,0.601265,0.0,0.799049,0.0,...,1,1,1,0,1,0,0,0,0,0


In [172]:
#Finally, we'll export this to a new .csv file which we can use as a basis for modelling going forward.
filename = 'jobs_clean.csv'
jobs.to_csv(filename)